# Exploration and Preprocessing of VGP database

This notebook preprocessing site level data from the VGP database compilation, conducts associated statistical tests and make some visualizations of data at the study level.

## Import scientific Python libraries

Import scipy python libraries as well as functions written for the project within vgptools.

In [ ]:
import os
import numpy as np
import pandas as pd
from pmagpy import ipmag, pmag
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cartopy.crs as ccrs
from scipy import optimize
import seaborn as sns

from vgptools.auxiliar import (get_files_in_directory, spherical2cartesian, 
                               cartesian2spherical, GCD_cartesian, 
                               print_pole_statistics, test_fishqq, 
                               statistical_tests, summary_figure,
                               invert_polarity, Plot_VgpsAndSites)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

## Gather study files

We retrieve all the spreadsheet files corresponding to different studies for which site level data are compiled. 

In [ ]:
current_path = os.getcwd()
data_path_VGP = current_path + '/data/vgp_database'
files_names = get_files_in_directory(data_path_VGP)

xlsx_file_names = [os.path.splitext(os.path.basename(open(file,'r').name))[0] for file in files_names if file.endswith('.xlsx')] 
paths = [file for file in files_names if file.endswith('.xlsx')] 
df_files = pd.DataFrame({'path': paths,  'name_xlsx': xlsx_file_names})
df_files[['name_xlsx']]

## Single study inspection

In order to understand what is going on in one single site, we can conduct statistical tests and visualize the data. 

In [ ]:
file_idx = 21

#### Separate the *.xlsx file into two different DFs, `df_vgps` and `df_poles`. 
Note: the the number of lines to be skipped is hardcoded

In [ ]:
def split_datasheet (df_files, file_idx):
    """
    Reads in datasheets and splits them into pole and vgp collections to be filtered, collated and compiled.
    Input: standard vgp datasheet (as described in datasheet template)
    Output: separate dataframes comprised of the study-level poles and site-level vgps extracted from the datasheet
    """
    df = pd.read_excel(df_files['path'][file_idx]) #, skip_blank_lines=True
    df_poles = pd.read_excel(df_files['path'][file_idx], 
                             skiprows = df[df.iloc[:,0]=='Study level data'].index[0]+2,
                             nrows  = df[df.isnull().all(1)].index[1] -3)

    df_vgps = pd.read_excel(df_files['path'][file_idx], 
                            skiprows = df[df.iloc[:,0]=='Site level data'].index[0]+2)

    #cast columns
    df_vgps = df_vgps.astype({'in_study_pole': int,
                              "slat":float, "slon":float, "dec":float, "inc":float,
                              "VGP_lat":float, "VGP_lon":float
                             })
    df_poles = df_poles.astype({'N': int,
                              "slat":float, "slon":float, "dec":float, "inc":float,
                              "Plat":float, "Plon":float})
    return (df_poles, df_vgps)

df_poles, df_vgps = split_datasheet(df_files, file_idx)

## We then proceed to populate the VGP DataFrame (`df_vgps`) following different criteria
 1. In a previous step, we have calculated the site coordinates of all studies in which these coordinates were not reported, but the Dec/Inc and Plat/Plon were. 
 2. When VGP coordinates are NOT reported in the original manuscript but the dec/inc and location of the sites was, we calculate them, and take them as face value.

In [ ]:
# df_vgps['VGP_lon'] = df_vgps.apply(lambda row: pmag.dia_vgp(row.dec, row.inc, 1, row.slat, row.slon)[0] if (np.isnan(row.VGP_lon) & (~np.isnan(row.dec) & ~np.isnan(row.slat))) else row.VGP_lon, axis =1) 
# df_vgps['VGP_lat'] = df_vgps.apply(lambda row: pmag.dia_vgp(row.dec, row.inc, 1, row.slat, row.slon)[1] if (np.isnan(row.VGP_lat) & (~np.isnan(row.dec) & ~np.isnan(row.slat))) else row.VGP_lat, axis =1) 

## Check polarity of VGPs against directions
 3. Some sites report the backward VGP (so that the VGPs are in the same hemisphere/closer to the principal component). We proceed to check polarity of VGPs against directions. To do that, we recalculate the vgps from the original Dec/Inc.

In [ ]:
#First we calculate for the entire dataframe the vgps if there is dec/inc and slat/slon values
def recalc_vgps(df_vgps):
    df_vgps['VGP_lon_recalc'] = df_vgps.apply(lambda row: pmag.dia_vgp(row.dec, row.inc, 1, row.slat, row.slon)[0], axis =1)
    df_vgps['VGP_lat_recalc'] = df_vgps.apply(lambda row: pmag.dia_vgp(row.dec, row.inc, 1, row.slat, row.slon)[1], axis =1)
    return df_vgps

df_vgps = recalc_vgps(df_vgps)

 4. Check distance (`df_vgps['GCD_vgps']`) between the reported VGPs and the recalculated from the directions.

In this step we fill the column `df_vgps['coherent_vgps']` with the following tags: 
- 'spurious' if inconsistent combination of site coordinates + dec/inc + vgp data (+- 4 degrees away from the reported or its backward)
- 'coherent' if correct
- 'inverted' if inverted

In [ ]:
def check_coherence_vgps(df_vgps):

    df_vgps['GCD_vgps'] = df_vgps.apply(lambda row: GCD_cartesian(spherical2cartesian([np.radians(row.VGP_lat), np.radians(row.VGP_lon)]), spherical2cartesian([np.radians(row.VGP_lat_recalc), np.radians(row.VGP_lon_recalc)])), axis=1)

    # False if Spurious, True if correct, nan if inverted
    df_vgps['coherent_vgps'] = df_vgps.apply(lambda row: 'spurious' if (row.GCD_vgps > np.radians(4) and row.GCD_vgps < np.radians(176)) else ('coherent' if row.GCD_vgps < np.radians(4) else 'inverted' if row.GCD_vgps > np.radians(176) else np.nan ), axis =1) #True if it is ok, nan
    
    return df_vgps
df_vgps = check_coherence_vgps(df_vgps)

### Catch some exceptions:
 - Missing slat/slon and/or dec/in where no vgp is reported
 - Missing dec/inc and/or vgp where no site coordinates are reported; cannot calculate site locations.
 - Inconsistent combination of site coordinates + dec/inc + vgp
 - Recalculated VGP was inverted. 

In [ ]:
def verbose(df_vgps):
    
    if not df_vgps[(df_vgps['slat'].isna() | df_vgps['slon'].isna())].empty:
        print (f" => Missing slat/slon from sites ('name'): {df_vgps[(df_vgps['slat'].isna() | df_vgps['slon'].isna()) ].name.tolist()}")
        print (f"")

    if not df_vgps[(df_vgps['dec'].isna() | df_vgps['inc'].isna())].empty:
        print (f" => Missing dec/inc from sites ('name'): {df_vgps[(df_vgps['dec'].isna() | df_vgps['inc'].isna()) ].name.tolist()}")
        print (f"")

    if not df_vgps[(df_vgps['VGP_lat'].isna() | df_vgps['VGP_lon'].isna())].empty:
        print (f" => Missing reported VGPs from sites ('name'): {df_vgps[(df_vgps['VGP_lat'].isna() | df_vgps['VGP_lon'].isna()) ].name.tolist()}")
        print (f"")

    if not df_vgps[(df_vgps['dec'].isna() & df_vgps['slat'].isna())].empty:    
        print (f" => Missing slat/slon and/or dec/inc from sites ('name'): {df_vgps[(df_vgps['dec'].isna() & df_vgps['slat'].isna())].name.tolist()} where no vgp is reported; cannot calculate vgp")
        print (f"")

    if not df_vgps[df_vgps['coherent_vgps'] == 'spurious'].empty:
        print (f" => Inconsistent combination of site coordinates + dec/inc + vgp data from site(s) {df_vgps[df_vgps['coherent_vgps'] == 'spurious'].name.tolist()}")
        print (f"")

    if not df_vgps[df_vgps['coherent_vgps'] == 'inverted'].empty:
        print (f" => inverted vgp from sites ('name'): {df_vgps[df_vgps['coherent_vgps'] == 'inverted'].name.tolist()}")
        print (f"")

    if not df_vgps[df_vgps['coherent_vgps'] == 'coherent'].empty:
        print (f" => Coherent dec/inc in sites ('name'): {df_vgps[df_vgps['coherent_vgps'] == 'coherent'].name.tolist()}")
        print (f"")
        
verbose(df_vgps)

### Summarize site level data

For each reported pole from the selected manuscript, we iterate through the constituent site-level data and: 
1) cast all vgps into a common polarity and re-compute the Fisher mean paleomagnetic pole
2) plot the site locations, vgps, and the results of reversal and Fisher distribution tests

In [ ]:
#Groupby iterates through DFs (i) 'grouped by' the column of interest 
misf1= []
misf2= []
studies= []
pps= []

def process_study(df_vgps,file_idx):
    for pole, df_pole in df_vgps.groupby('in_study_pole'):

        # value represent the index and i represent the DF grouped by the variable of interest  
        if pole != 0: #discards vgps discarded by authors

            print(f"==>Analyzing pole {pole} ({df_files.name_xlsx[file_idx]}).")
            print('')
            #try:

            directions_block = ipmag.make_di_block(df_pole['dec'].tolist(), 
                                                   df_pole['inc'].tolist(),
                                                   unit_vector=False)
            di_mode1, di_mode2 = pmag.separate_directions(di_block=directions_block)

            vgp_recalc_block = ipmag.make_di_block(df_pole['VGP_lon_recalc'].tolist(), 
                                                   df_pole['VGP_lat_recalc'].tolist(),
                                                   unit_vector=False)
            #split recalculated vgp population into polarities             
            vgp_mode1, vgp_mode2 = pmag.separate_directions(di_block=vgp_recalc_block)
            merged_vgps = invert_polarity(vgp_mode1, vgp_mode2)
            vgp_mean_recomputed = ipmag.fisher_mean(di_block = merged_vgps)

            # calculate the Fisher mean of reported VGPs
            df_pole['VGP_lat'] = np.where(df_pole['VGP_lat'].isna(), df_pole['VGP_lat_recalc'], df_pole['VGP_lat'])
            df_pole['VGP_lon'] = np.where(df_pole['VGP_lon'].isna(), df_pole['VGP_lon_recalc'], df_pole['VGP_lon'])
            df_pole['vgp_lat_NH'] = np.where(df_pole['VGP_lat'] < 0, -df_pole['VGP_lat'], df_pole['VGP_lat'])
            df_pole['vgp_lon_NH'] = np.where(df_pole['VGP_lat'] < 0,(df_pole['VGP_lon'] - 180.) % 360., df_pole['VGP_lon'])             
            vgp_mean = ipmag.fisher_mean(dec = df_pole['vgp_lon_NH'].tolist(), inc = df_pole['vgp_lat_NH'].tolist())

            #reported pole
            reported_pole = df_poles[df_poles['pole'] == pole]

            study_number = file_idx
            study_folder = './study_summaries' + '/' + str(study_number)
            if not os.path.exists(study_folder):
                os.mkdir(study_folder)

            pole_number = pole
            pole_folder = study_folder + '/' + str(pole_number)
            if not os.path.exists(pole_folder):
                os.mkdir(pole_folder)

            pole_summary = print_pole_statistics(reported_pole, 
                                                 vgp_mean, 
                                                 vgp_mean_recomputed)

            with open(pole_folder + '/pole_summary.tex','w') as tf:
                tf.write(pole_summary.to_latex())

            stat_test_results = statistical_tests(di_mode1, di_mode2, merged_vgps, 
                                                  study_number=file_idx, 
                                                  pole_number=pole, 
                                                  save_folder=pole_folder)

            with open(pole_folder + '/stat_test.tex','w') as tf:
                tf.write(stat_test_results.to_latex())

            summary_figure(df_pole, vgp_mode1, vgp_mode2, 
                           reported_pole, vgp_mean, pole_folder)

    #         except: #could be expanded!
    #             print(f" DEBUG POLE (index): {file_idx}, in_situ_pole: {pole}") 
            
            
            study_name = str(df_files.name_xlsx[file_idx])
            study_name = study_name.replace('_', ' ')
            pole_local_folder = './' + str(study_number) + '/' + str(pole_number)
            tex_file = open('./study_summaries/SI_study_summary.tex', 'a')
            tex_file.write('\n')
            
            if pole_number == 1:
                tex_file.write('\section{' + study_name + '}')
                tex_file.write('\n')
                tex_file.write('\subsection{Pole ' + str(pole) + '}')
            if pole_number > 1:
                tex_file.write('\subsection{Pole ' + str(pole) + '}')
            tex_file.write('\n')
            tex_file.write('\input{' + pole_local_folder + '/pole_summary.tex' + '}')
            tex_file.write('\n')
            tex_file.write('\input{' + pole_local_folder + '/stat_test.tex' + '}')
            tex_file.write('\n')
            tex_file.write('\\begin{figure}[H]')
            tex_file.write('\n')
            tex_file.write('\centering')
            tex_file.write('\n')
            tex_file.write('\includegraphics[width=5 in]{' + pole_local_folder + '/pole_summary.png}')
            tex_file.write('\n')
            tex_file.write('\caption{Summary of data from study ' + str(file_idx) + ': ' + study_name + '}')
            tex_file.write('\n')
            tex_file.write('\end{figure}')
            tex_file.write('\n')
            tex_file.close()
   
            print('')
            plt.pause(10)
            
process_study(df_vgps,file_idx)

# Iterate through all the files to catch exceptions:

In [ ]:
for file_idx in range(0,31):
    print(f'')
    print(f'========================= NEW POLE : {df_files.name_xlsx[file_idx]} ({file_idx}) =======================')
    
    df_poles, df_vgps = split_datasheet(df_files, file_idx)
    df_vgps = recalc_vgps(df_vgps)
    df_vgps = check_coherence_vgps(df_vgps)
    verbose(df_vgps)
    process_study(df_vgps,file_idx)

In [ ]:
studies, pps, misf, froms = [], [], [], []
for file_idx in range(len(df_files) -1):
    
    print(f'')
    print(f'========================= NEW POLE : {df_files.name_xlsx[file_idx]} ({file_idx}) =======================')
    
    df_poles, df_vgps = split_datasheet(df_files, file_idx)
    
    if df_vgps.empty: continue
        
    df_vgps = recalc_vgps(df_vgps)
    
    df_vgps = check_coherence_vgps(df_vgps)
   
    verbose(df_vgps)
    
    #Groupby iterates through DFs (i) 'grouped by' the column of interest 
    
    

    
    for pole, df_pole in df_vgps.groupby('in_study_pole'):

        # value represent the index and i represent the DF grouped by the variable of interest  
        if pole != 0: #discards vgps discarded by authors

            print(f"==>Analyzing pole {pole} ({df_files.name_xlsx[file_idx]}).")
            try:

                #split recalculated vgp population into different polarities by comparison against principal component axis             
                vgp_recalc_block = list(zip(df_pole['VGP_lon_recalc'].tolist(), df_pole['VGP_lat_recalc'].tolist()))            
                mode1, mode2 = pmag.separate_directions(di_block=vgp_recalc_block)

                #reversal test            
                reversal_test(mode1, mode2)    

                #Bayesian reversal Test
                P = bayes_probability_heslop(mode1, mode2)
                if P != 9999: print(f"Bayesian reversal test (Heslop & Roberts, 2018) indicates: {bayes_support(P)}")            

                #invert one polarity population, merge populations and test whether a distribution is Fisherian
                merged = invert_polarity(mode1, mode2)
                test_fishqq(merged)

                # calculate the Fisher mean of reported VGPs to test reproducibility (we first fill the 'reported' if empty)
                df_pole['VGP_lat'] = np.where(df_pole['VGP_lat'].isna(), df_pole['VGP_lat_recalc'], df_pole['VGP_lat'])
                df_pole['VGP_lon'] = np.where(df_pole['VGP_lon'].isna(), df_pole['VGP_lon_recalc'], df_pole['VGP_lon'])

                df_pole['vgp_lat_NH'] = np.where(df_pole['VGP_lat'] < 0, -df_pole['VGP_lat'], df_pole['VGP_lat'])
                df_pole['vgp_lon_NH'] = np.where(df_pole['VGP_lat'] < 0,(df_pole['VGP_lon'] - 180.) % 360., df_pole['VGP_lon'])             

                vgp_mean_recomputed = ipmag.fisher_mean(di_block = merged)
                vgp_mean = ipmag.fisher_mean(dec = df_pole['vgp_lon_NH'].tolist(), inc = df_pole['vgp_lat_NH'].tolist())

                #Reported pole
                pp = df_poles[df_poles['pole'] == pole]

                print_pole_statistics(pp, vgp_mean, vgp_mean_recomputed)

                #di_block = ipmag.make_di_block(df_pole['VGP_lon'].tolist(),df_pole['VGP_lat'].tolist())

                # Plot sites and directions
                Plot_VgpsAndSites(df_pole, pp, vgp_mean, mode1, mode2) 
                
                
                studies.append(df_files.name_xlsx[file_idx])
                pps.append(pole)
                misf.append(pmag.angle([vgp_mean['dec'], vgp_mean['inc']], [pp.iloc[0]['Plon'], pp.iloc[0]['Plat']])[0])
                froms.append('reported')
                
                studies.append(df_files.name_xlsx[file_idx])
                pps.append(pole)                
                misf.append(pmag.angle([vgp_mean_recomputed['dec'], vgp_mean_recomputed['inc']], [pp.iloc[0]['Plon'], pp.iloc[0]['Plat']])[0])
                froms.append('recalculated')
                
            except: #could be expanded!
                print(f" DEBUG POLE (index): {file_idx}, in_situ_pole: {pole}") 


In [ ]:
studies, pps, misf, froms = [], [], [], []
for file_idx in range(len(df_files) -1):
    
    print(f'')
    print(f'========================= NEW POLE : {df_files.name_xlsx[file_idx]} ({file_idx}) =======================')
    
    df_poles, df_vgps = split_datasheet(df_files, file_idx)
    
    if df_vgps.empty: continue
        
    df_vgps = recalc_vgps(df_vgps)
    
    df_vgps = check_coherence_vgps(df_vgps)
   
    #verbose(df_vgps)
    
    #Groupby iterates through DFs (i) 'grouped by' the column of interest 
    
    

    
    for pole, df_pole in df_vgps.groupby('in_study_pole'):

        # value represent the index and i represent the DF grouped by the variable of interest  
        if pole != 0: #discards vgps discarded by authors

            print(f"==>Analizing pole {pole} ({df_files.name_xlsx[file_idx]}).")
          
            try:
                #split recalculated vgp population into different polarities by comparison against principal component axis             
                vgp_recalc_block = list(zip(df_pole['VGP_lon_recalc'].tolist(), df_pole['VGP_lat_recalc'].tolist()))            
                mode1, mode2 = pmag.separate_directions(di_block=vgp_recalc_block)

                #reversal test            
                # reversal_test(mode1, mode2)    

                #Bayesian reversal Test
                # P = bayes_probability_heslop(mode1, mode2)
                # if P != 9999: print(f"Bayesian reversal test (Heslop & Roberts, 2018) indicates: {bayes_support(P)}")            

                #invert one polarity population, merge populations and test whether a distribution is Fisherian
                merged = invert_polarity(mode1, mode2)
                # test_fishqq(merged)

                # calculate the Fisher mean of reported VGPs to test reproducibility (we first fill the 'reported' if empty)
                df_pole['VGP_lat'] = np.where(df_pole['VGP_lat'].isna(), df_pole['VGP_lat_recalc'], df_pole['VGP_lat'])
                df_pole['VGP_lon'] = np.where(df_pole['VGP_lon'].isna(), df_pole['VGP_lon_recalc'], df_pole['VGP_lon'])

                df_pole['vgp_lat_NH'] = np.where(df_pole['VGP_lat'] < 0, -df_pole['VGP_lat'], df_pole['VGP_lat'])
                df_pole['vgp_lon_NH'] = np.where(df_pole['VGP_lat'] < 0,(df_pole['VGP_lon'] - 180.) % 360., df_pole['VGP_lon'])             

                vgp_mean_recomputed = ipmag.fisher_mean(di_block = merged)
                vgp_mean = ipmag.fisher_mean(dec = df_pole['vgp_lon_NH'].tolist(), inc = df_pole['vgp_lat_NH'].tolist())

                #Reported pole
                pp = df_poles[df_poles['pole'] == pole]

                #print_pole_statistics(pp, vgp_mean, vgp_mean_recomputed)

                #di_block = ipmag.make_di_block(df_pole['VGP_lon'].tolist(),df_pole['VGP_lat'].tolist())

                # Plot sites and directions
                # Plot_VgpsAndSites(df_pole, pp, vgp_mean, mode1, mode2) 

                # if not (len(vgp_mean_recomputed)!=0 & len(vgp_mean)!=0): 

                
                misf.append(pmag.angle([vgp_mean['dec'], vgp_mean['inc']], [pp.iloc[0]['Plon'], pp.iloc[0]['Plat']])[0])
                froms.append('reported')
                studies.append(df_files.name_xlsx[file_idx])
                pps.append(pole)
                
                misf.append(pmag.angle([vgp_mean_recomputed['dec'], vgp_mean_recomputed['inc']], [pp.iloc[0]['Plon'], pp.iloc[0]['Plat']])[0])
                studies.append(df_files.name_xlsx[file_idx])
                pps.append(pole)                                
                froms.append('recalculated')
                    
            except: #could be expanded!
                print(f" DEBUG POLE (index): {file_idx}, in_situ_pole: {pole}") 
            

In [ ]:

dictionary= {'Study': studies, 'in_study_pole': pps, 'Misfit': misf, 'From' : froms}
df_misfits = pd.DataFrame(dictionary)

df_misfits = df_misfits[~df_misfits['Misfit'].isna()]

df_misfits['Misfit'] = np.where(df_misfits['Misfit'] > 90, 180 - df_misfits['Misfit'], df_misfits['Misfit'])
df_misfits

In [ ]:
sns.set_theme(style="whitegrid")
sns.set(rc={'figure.figsize':(16,8)})
ax = sns.barplot(x="Study", y= "Misfit", hue="in_study_pole", data=df_misfits[df_misfits['From'] == 'reported'])
ax.tick_params(axis='x', rotation=90)
ax.set(title = "Distance from the reported Paleopole and the reported VGPs")
#ax.xaxis.get_label().set_fontsize(20)

In [ ]:
sns.set_theme(style="whitegrid")
sns.set(rc={'figure.figsize':(16,8)})
ax = sns.barplot(x="Study", y= "Misfit", hue="in_study_pole", data=df_misfits[df_misfits['From'] == 'recalculated'])
ax.tick_params(axis='x', rotation=90)
ax.set(title = "Distance from the reported Paleopole and the recalculated VGPs")
#ax.xaxis.get_label().set_fontsize(20)